In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

# Load and preprocess data
data = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\IQR.csv',parse_dates=['Date'], index_col='Date')
data = data['Price'].values
data = data.astype('float32')

# Split data into training and test sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:len(data)]

# Normalize data
mean = train.mean()
std = train.std()
train = (train - mean) / std
test = (test - mean) / std

# Create dataset for training
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size - 1):
        X.append(data[i:(i + window_size)])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 10
X_train, y_train = create_dataset(train, window_size)
X_test, y_test = create_dataset(test, window_size)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
model = keras.Sequential()
model.add(layers.LSTM(4, input_shape=(window_size, 1)))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model on GPU
with tf.device('/device:GPU:0'):
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions
train_predict = (train_predict * std) + mean
y_train = (y_train * std) + mean
test_predict = (test_predict * std) + mean
y_test = (y_test * std) + mean

# Calculate root mean squared error
train_score = np.sqrt(np.mean((train_predict - y_train) ** 2))
print('Train Score: %.2f RMSE' % train_score)
test_score = np.sqrt(np.mean((test_predict - y_test) ** 2))
print('Test Score: %.2f RMSE' % test_score)


Epoch 1/100
144/144 - 4s - loss: 0.6408 - val_loss: 0.1419 - 4s/epoch - 30ms/step
Epoch 2/100
144/144 - 1s - loss: 0.1340 - val_loss: 0.0534 - 890ms/epoch - 6ms/step
Epoch 3/100
144/144 - 1s - loss: 0.0612 - val_loss: 0.0312 - 692ms/epoch - 5ms/step
Epoch 4/100
144/144 - 0s - loss: 0.0344 - val_loss: 0.0179 - 476ms/epoch - 3ms/step
Epoch 5/100
144/144 - 0s - loss: 0.0199 - val_loss: 0.0132 - 473ms/epoch - 3ms/step
Epoch 6/100
144/144 - 1s - loss: 0.0142 - val_loss: 0.0120 - 501ms/epoch - 3ms/step
Epoch 7/100
144/144 - 1s - loss: 0.0119 - val_loss: 0.0115 - 672ms/epoch - 5ms/step
Epoch 8/100
144/144 - 1s - loss: 0.0103 - val_loss: 0.0112 - 945ms/epoch - 7ms/step
Epoch 9/100
144/144 - 1s - loss: 0.0092 - val_loss: 0.0108 - 1s/epoch - 7ms/step
Epoch 10/100
144/144 - 1s - loss: 0.0082 - val_loss: 0.0104 - 990ms/epoch - 7ms/step
Epoch 11/100
144/144 - 1s - loss: 0.0075 - val_loss: 0.0101 - 884ms/epoch - 6ms/step
Epoch 12/100
144/144 - 1s - loss: 0.0069 - val_loss: 0.0101 - 900ms/epoch - 6ms

In [7]:
from datetime import datetime, timedelta

# Define a function to make a prediction for a future date
def predict_price(model, last_date, future_date):
    # Calculate the number of days between the last date in the data and the future date
    days = (future_date - last_date).days

    # Initialize the input data with the last known price
    input_data = data.iloc[-1]['Price']

    # Make predictions for each day up to the future date
    for i in range(days):
        input_data = model.predict(input_data.reshape(1, -1))

    # Return the predicted price for the future date
    return input_data[0][0]

# Get the last date in the data
last_date = data.index[-1]

# Specify the future date for which to make a prediction
future_date = datetime.strptime(input('Enter future date (YYYY-MM-DD): '), '%Y-%m-%d')

# Make a prediction for the future date using the trained model
predicted_price = predict_price(model, last_date, future_date)

# Print the predicted price
print(f'Predicted price for {future_date.date()}: {predicted_price}')


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [10]:
from datetime import datetime
import numpy as np
data = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\IQR.csv',parse_dates=['Date'], index_col='Date')
data = data['Price'].values
data = data.astype('float32')

def predict_price(date):
    # Convert date to datetime object
    date = datetime.strptime(date, '%Y-%m-%d')
    
    # Check if date is within data range
    if date < data.index[0] or date > data.index[-1]:
        print('Date is out of range!')
        return
    
    # Get data for given date
    date_data = data.loc[date]
    
    # Normalize data
    date_data = (date_data - mean) / std
    
    # Create dataset for prediction
    X_date = np.array([date_data])
    X_date = np.reshape(X_date, (X_date.shape[0], X_date.shape[1], 1))
    
    # Make prediction
    prediction = model.predict(X_date)
    
    # Inverse transform prediction
    prediction = (prediction * std) + mean
    
    return prediction[0][0]

# Example usage
date = '2023-08-05'
prediction = predict_price(date)
print(f'Predicted price for {date}: {prediction:.2f}')


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

# Load and preprocess data
df = pd.read_csv('E:\Excelr-P269-Internship-Project-Group2-OIl-Price-Prediction-And-Forecasting-Using-Python\Dataset\IQR.csv', parse_dates=['Date'], index_col='Date')
data = df['Price'].values
data = data.astype('float32')

# Split data into training and test sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:len(data)]

# Normalize data
mean = train.mean()
std = train.std()
train = (train - mean) / std
test = (test - mean) / std

# Create dataset for training
def create_dataset(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size - 1):
        X.append(data[i:(i + window_size)])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 10
X_train, y_train = create_dataset(train, window_size)
X_test, y_test = create_dataset(test, window_size)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Build LSTM model
model = keras.Sequential()
model.add(layers.LSTM(4, input_shape=(window_size, 1)))
model.add(layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train model on GPU
with tf.device('/device:GPU:0'):
    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions
train_predict = (train_predict * std) + mean
y_train = (y_train * std) + mean
test_predict = (test_predict * std) + mean
y_test = (y_test * std) + mean

# Calculate root mean squared error
train_score = np.sqrt(np.mean((train_predict - y_train) ** 2))
print('Train Score: %.2f RMSE' % train_score)
test_score = np.sqrt(np.mean((test_predict - y_test) ** 2))
print('Test Score: %.2f RMSE' % test_score)


Epoch 1/100
144/144 - 5s - loss: 0.5627 - val_loss: 0.1713 - 5s/epoch - 33ms/step
Epoch 2/100
144/144 - 1s - loss: 0.1376 - val_loss: 0.0436 - 835ms/epoch - 6ms/step
Epoch 3/100
144/144 - 1s - loss: 0.0520 - val_loss: 0.0282 - 830ms/epoch - 6ms/step
Epoch 4/100
144/144 - 1s - loss: 0.0345 - val_loss: 0.0191 - 837ms/epoch - 6ms/step
Epoch 5/100
144/144 - 1s - loss: 0.0242 - val_loss: 0.0146 - 809ms/epoch - 6ms/step
Epoch 6/100
144/144 - 1s - loss: 0.0178 - val_loss: 0.0116 - 806ms/epoch - 6ms/step
Epoch 7/100
144/144 - 1s - loss: 0.0136 - val_loss: 0.0099 - 798ms/epoch - 6ms/step
Epoch 8/100
144/144 - 1s - loss: 0.0107 - val_loss: 0.0088 - 833ms/epoch - 6ms/step
Epoch 9/100
144/144 - 1s - loss: 0.0087 - val_loss: 0.0081 - 858ms/epoch - 6ms/step
Epoch 10/100
144/144 - 1s - loss: 0.0073 - val_loss: 0.0079 - 835ms/epoch - 6ms/step
Epoch 11/100
144/144 - 1s - loss: 0.0064 - val_loss: 0.0078 - 880ms/epoch - 6ms/step
Epoch 12/100
144/144 - 1s - loss: 0.0058 - val_loss: 0.0077 - 828ms/epoch - 

In [12]:
def predict_price(date):
    # Convert date to datetime object
    date = datetime.strptime(date, '%Y-%m-%d')
    
    # Check if date is within data range
    if date < data.index[0] or date > data.index[-1]:
        print('Date is out of range!')
        return
    
    # Get data for given date
    date_data = data.loc[date]
    
    # Normalize data
    date_data = (date_data - mean) / std
    
    # Create dataset for prediction
    X_date = np.array([date_data])
    X_date = np.reshape(X_date, (X_date.shape[0], X_date.shape[1], 1))
    
    # Make prediction
    prediction = model.predict(X_date)
    
    # Inverse transform prediction
    prediction = (prediction * std) + mean
    
    return prediction[0][0]

# Example usage
date = '2023-08-05'
prediction = predict_price(date)
print(f'Predicted price for {date}: {prediction:.2f}')

AttributeError: 'numpy.ndarray' object has no attribute 'index'